In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **시나리오**

1. input -> OOOO은 얼마야?(금융 관련 용어에 대한 질문)
2. 해당 input의 embedding 변환 (finBERT 등 활용)
3. 문서 embedding의 문서는 외부 금융용어사전 크롤링
4. embedding간의 유사도 측정(아마 cosine similiarity
5. 최종 답변 생성(gpt 사용)

## 관련 라이브러리 다운

In [6]:
# langchain
!pip install langchain
!pip install -U langchain-openai
!pip install -U langchain-community
!pip install faiss-gpu
!pip install langchainhub

In [3]:
# crawling
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 36.1 MB/s eta 0:00:00
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:12 http://security.ubuntu.

In [7]:
# langchain

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain_community.vectorstores import Chroma, FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [8]:
# web crawling

from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys
import numpy as np
import pandas as pd
import re
import os
import pprint
from selenium import webdriver
import time
import urllib.request
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# 기타

import openai
from openai import OpenAI
from typing import List, Optional
from pydantic import BaseModel

In [ ]:
## gpt key = sk-proj-5vrBpk9gQ4bYF8OljiDST3BlbkFJ5Gz2QGqHc2aW6CYKo8w0

## 예시 코드

In [17]:
## preprocessing
def text_preprocess(text):
    cleaned_text = re.sub(r"(<span class='quot[0-9]'>|\n\r\n|</dl>|</dd>|<dl>|<dt>|</span>|<br/>|<br />|\[.*?\]|([^0-9가-힣A-Za-z.]))", "", text)
    return cleaned_text

# web crawling(fine_word_data)
def create_word_dict(num):

  global words_list, definitions_list

  word_address = 'https://fine.fss.or.kr/fine/fnctip/fncDicary/list.do?menuNo=900021&pageIndex=' + num + '&src=&kind=&searchCnd=1&searchStr='
  requested = requests.get(word_address, 'html.parser')
  soup = BeautifulSoup(requested.content)

  words = soup.select('#content > div.bd-list.result-list > dl > dt') # word
  definitions =  soup.select('#content > div.bd-list.result-list > dl > dd') # defin

  words_list = []
  definitions_list = []

  for word, definition in zip(words, definitions):
    words_list.append(text_preprocess(word.get_text()))
    definitions_list.append(definition.get_text())

  return words_list, definitions_list

In [21]:
# dictionary

word_dict = []
definition_dict = []

for i in range(1,55):
  word, definition = create_word_dict(str(i))
  word_dict.extend(word)
  definition_dict.extend(definition)

In [23]:
# 오픈키 정의
openai_api_key = "sk-proj-5vrBpk9gQ4bYF8OljiDST3BlbkFJ5Gz2QGqHc2aW6CYKo8w0"

# embedding
embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)

# 문서 생성(Create Document)
class Document():
    def __init__(self, content: str):
        self.page_content = content

docs = [f"{x} 정의 : {y}" for x, y in zip(words_list, definitions_list)]

# list -> Document 객체의 리스트
class Document(BaseModel):
    page_content: str
    metadata: Optional[dict] = None

docs_objects = [Document(page_content=doc, metadata={"source": "example"}) for doc in docs]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [24]:
# embedding + create vectorstore
def create_vector_store(docs: List[Document], embedding_model):
    vectorstore = FAISS.from_documents(documents=docs_objects, embedding=embedding)
    return vectorstore

vectorstore = create_vector_store(docs, embedding)

# retriever & prompt
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

# llm
llm = ChatOpenAI(openai_api_key = openai_api_key, model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# rag
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Q&A
question = "환매조건부매매가 뭐야?"
response = rag_chain.invoke(question)

print(f"[QUERY]\n{question}\n")
print(f"[ANSWER]\n{response}")

[HUMAN]
환매조건부매매가 뭐야?

[AI]
환매조건부매매는 가장매매와 관련된 매매로, 동일인이 동일 종목에 대해 매수와 매도주문을 같은 시기에 하는 행위를 말합니다. 이는 시세조종 행위로 금지되어 현실수요를 반영하지 않는 거래량 및 가격변동을 초래할 수 있습니다. 환매조건부매매는 거래 성황을 오인하도록 하는 목적으로 금지되는 행위입니다.


In [28]:
# api_key
client = OpenAI(api_key="sk-proj-5vrBpk9gQ4bYF8OljiDST3BlbkFJ5Gz2QGqHc2aW6CYKo8w0")

# changing answer(response -> simplify answer)
def simplify_definition(definition: str) -> str:

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"다음 정의를 모르는 사람의 입장에서 이해하기 쉽고 너무 길지 않는 용어로 재작성해 주세요:\n\n정의: {definition}\n\n재작성된 정의:"}
    ]


    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # model(gpt4 가능)
        messages=messages,
        max_tokens=150,
        temperature=0.5
    )
    simplified_definition = response.choices[0].message.content.strip()
    return simplified_definition

result = simplify_definition(response)

print("원본 정의:", response)
print("간단한 정의:", result)

원본 정의: 환매조건부매매는 가장매매와 관련된 매매로, 동일인이 동일 종목에 대해 매수와 매도주문을 같은 시기에 하는 행위를 말합니다. 이는 시세조종 행위로 금지되어 현실수요를 반영하지 않는 거래량 및 가격변동을 초래할 수 있습니다. 환매조건부매매는 거래 성황을 오인하도록 하는 목적으로 금지되는 행위입니다.
간단한 정의: 환매조건부매매는 동일한 사람이 동일한 주식을 동시에 사고 팔기 위한 거래 방식입니다. 이는 시장을 조작하여 가격과 거래량을 부풀릴 수 있으므로 금지되어 있습니다. 이 방식은 거래를 가장 잘하고 있다고 오해를 일으키는 목적으로 사용되지 않아야 합니다.
